In [1]:
from os import walk
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle
from utils.load_data_raw import DataGenerator_raw, load_raw_all_h5
from utils.dataset_split import create_split

NUM_WORKER = 4

Using TensorFlow backend.


In [2]:
def plot_history(hist, val_hist, metric_str):
    """Plot train history."""
    plt.plot(hist)
    plt.plot(val_hist)
    plt.title('Model loss ('+metric_str+')')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
def model_eval(model, history, part_test=[], batch_size=1024, feature_data=[], target_data=[], par=[]):
    """Evaluate a specified model. Show model topology, training history and loss on test data set."""
    # Get metrics from history dict
    metrics = list(history.keys())
    v_met = [x for x in metrics if 'val' in x]
    n = len(v_met)
    metrics = metrics[n:]
    
    # Show model/net topology
    model.summary()

    # Evaluate model
    if part_test:
        params = {'dim': feature_data.shape[1],
          'batch_size': batch_size,
          'feature_data': feature_data.values,
          'target_data' : target_data.values,
          'shuffle': False,
          'n_frames': par['nFrames'],
          'n_angles': par['nAngles']
         }
        print('Model Evaluation:')
        test_batch_generator = DataGenerator_raw(part_test, **params)
        score = model.evaluate_generator(test_batch_generator, verbose=1, use_multiprocessing=True, workers=NUM_WORKER)
        for i, m in enumerate(metrics):
            print('Test '+m+':', score[i])
    else:
        test_batch_generator = 0

    # Plot train history
    for j in range(n):
        plot_history(history[metrics[j]], history[v_met[j]], metrics[j])
        
    return test_batch_generator

def open_json(dir, filename):
    """Open json file abd return its contents."""
    with open(dir + filename) as json_file:
        content = json.load(json_file)
    return content

In [3]:
model_dir = '../data/models_trained/'
# load filelist
filelist = []
for (dirpath, dirnames, filenames) in walk(model_dir):
    filelist.extend(filenames)
filelist.sort()

dset_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'
feat, targ, _, _, _, par = load_raw_all_h5(dset_dir)

In [5]:
h_flist = [x for x in filelist if 'history' in x]
pt_flist = [x for x in filelist if 'partition_test' in x]
m_flist = [x for x in filelist if x not in h_flist+pt_flist]

for i in range(len(m_flist)):
    s = ' -' if i<10 else '-'
    print(i, s, m_flist[i])
    print(i, s, h_flist[i])
    print(i, s, pt_flist[i])

0  - m064_maew_adam_bs1024_t-split_wraptest.h5
0  - m064_maew_adam_bs1024_t-split_wraptest_history.json
0  - m064_maew_adam_bs1024_t-split_wraptest_partition_test.json
1  - m128_mae_adam_bs1024_t-split_wraptest.h5
1  - m128_mae_adam_bs1024_t-split_wraptest_history.json
1  - m128_mae_adam_bs1024_t-split_wraptest_partition_test.json
2  - m128_maew_adam_bs1024_t-NHR006_toptest.h5
2  - m128_maew_adam_bs1024_t-NHR006_toptest_history.json
2  - m128_maew_adam_bs1024_t-NHR006_toptest_partition_test.json
3  - m128_maew_adam_bs1024_t-split_wraptest.h5
3  - m128_maew_adam_bs1024_t-split_wraptest_history.json
3  - m128_maew_adam_bs1024_t-split_wraptest_partition_test.json
4  - m256_mae_adam_bs1024_t-split_wraptest.h5
4  - m256_mae_adam_bs1024_t-split_wraptest_history.json
4  - m256_mae_adam_bs1024_t-split_wraptest_partition_test.json
5  - m256_maew_adam_bs1024_t-split_wraptest.h5
5  - m256_maew_adam_bs1024_t-split_wraptest_history.json
5  - m256_maew_adam_bs1024_t-split_wraptest_partition_test.jso

In [6]:
# load models, train history and test split IDs (partition['test'])
#m1_mae = load_model(model_dir + m_flist[0], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
m1_maew = load_model(model_dir + m_flist[0], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
m2_mae = load_model(model_dir + m_flist[1], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
m2_maew = load_model(model_dir + m_flist[3], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
m3_mae = load_model(model_dir + m_flist[4], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
m3_maew = load_model(model_dir + m_flist[5], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})


m1_tt = load_model(model_dir + m_flist[2], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})

#h1_mae  = open_json(model_dir, h_flist[0])
h1_maew = open_json(model_dir, h_flist[0])
h2_mae  = open_json(model_dir, h_flist[1])
h2_maew = open_json(model_dir, h_flist[3])
h3_mae  = open_json(model_dir, h_flist[4])
h3_maew = open_json(model_dir, h_flist[5])

h1_tt = open_json(model_dir, h_flist[2])


#pt1_mae = open_json(model_dir, pt_flist[0])
pt1_maew = open_json(model_dir, pt_flist[0])
pt2_mae = open_json(model_dir, pt_flist[1])
pt2_maew = open_json(model_dir, pt_flist[3])
pt3_mae = open_json(model_dir, pt_flist[4])
pt3_maew = open_json(model_dir, pt_flist[5])

pt1_tt = open_json(model_dir, pt_flist[2])

W1006 21:49:48.197153 139894133552960 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1006 21:49:48.225629 139894133552960 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1006 21:49:48.299826 139894133552960 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1006 21:49:48.301239 139894133552960 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.

## Test: custom vs. non-custom loss function

    Model 1: 64-32-1      m64_maew_wraptest   vs.  m64_mae_wraptest
    Model 2: 128-128-1    m128_maew_wraptest  vs.  m128_mae_wraptest
    Model 3: 256-128-1    m256_maew_wraptest  vs.  m256_mae_wraptest


In [ ]:
# Model 1 custom loss: m1_maew
tb_gen1_maew = model_eval(m1_maew, h1_maew, pt1_maew, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen1_maew.__getitem__(123)
#pred = m1_maew.predict_on_batch(X)

In [ ]:
# Model 1 mae loss: m1_mae
tb_gen1_mae = model_eval(m1_mae, h1_mae, pt1_mae, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen1_mae.__getitem__(123)
#pred = m1_mae.predict_on_batch(X)

In [ ]:
# Model 2 custom loss: m2_maew
tb_gen2_maew = model_eval(m2_maew, h2_maew, pt2_maew, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen2_maew.__getitem__(123)
#pred = m2_maew.predict_on_batch(X)

In [ ]:
# Model 2 mae loss: m2_mae
tb_gen2_mae = model_eval(m2_mae, h2_mae, pt2_mae, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen2_mae.__getitem__(123)
#pred = m2_mae.predict_on_batch(X)

In [ ]:
# Model 3 custom loss: m3_maew
tb_gen3_maew = model_eval(m3_maew, h3_maew, pt3_maew, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen3_maew.__getitem__(123)
#pred = m3_maew.predict_on_batch(X)

In [ ]:
# Model 3 mae loss: m3_mae
tb_gen3_mae = model_eval(m3_mae, h3_mae, pt3_mae, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen3_mae.__getitem__(123)
#pred = m3_mae.predict_on_batch(X)

## Test: Model/Net Topology
    Model 1:         128-128-1
    Model 1 + BN :   128-BN-128-BN-1 

In [ ]:
# Model 1: m1_tt
tb_gen1_tt = model_eval(m1_tt, h1_tt, pt1_tt, batch_size=1024, feature_data=feat, target_data=targ, par=par)

#predict on model
#X,y = tb_gen1_tt.__getitem__(123)
#pred = m1_tt.predict_on_batch(X)